In [43]:
import wordfreq
import numpy as np

In [44]:
with open("words/frequencies.csv", "rt") as f:
    word_frequencies = dict(line.split(",") for line in f.readlines())

probability = lambda w: word_frequencies[w]
information = lambda w: - np.log2(probability(w))

In [45]:
get_frequencies = lambda filepath: {
    word : information(word)
    for word in
    open(filepath).read().split("\n")
}

guesses = get_frequencies("words/guesses.txt")
answers = get_frequencies("words/answers.txt")

KeyError: 'aahed'

In [ ]:
import regex as re

answers = open("words/answers.txt").read()
guesses =open("words/guesses.txt").read()

def match(pattern, yellows, greys) -> list[str]:
    return [
        word for
        word in re.findall(pattern, answers) # matches greens
        if not any(char in word for char in greys) # doesn't contain greys
        and all(
            (char in word) # has yellows
            and (word[int(pos)-1] != char) # but not where they were yellow
            for char, pos in yellows
        )
    ]

In [ ]:
match(".ra.e", [("d", 0)], "cnbk")

['grade', 'trade']

In [ ]:
set(chain(*match(".....", ["u2", "s3", "l4", "s5"], "ertyioadkcn")))-set("slu")


{'b', 'f', 'h', 'm', 'p'}

In [ ]:
match(".....", ["u2", "s3", "l4", "s5", "m3", "p4"], "ertyioadkcn")

['slump']

In [ ]:
match(".a...", ["n1", "l1", "a3", "n4", "l4"], "ertyiosfc")

['banal']

In [ ]:
match("r...s", ["r2", "e4"], "canilop")

['rebus']

In [ ]:
re.findall(r"[eirwumhtovg]{5}", guesses)

['egger',
 'emeer',
 'emerg',
 'emmer',
 'emmet',
 'emmew',
 'emote',
 'emove',
 'emure',
 'ergot',
 'erugo',
 'etwee',
 'evert',
 'evhoe',
 'evite',
 'evohe',
 'ewhow',
 'gemot',
 'gighe',
 'gigot',
 'gigue',
 'gimme',
 'goeth',
 'goier',
 'gomer',
 'grege',
 'grego',
 'greve',
 'griot',
 'grith',
 'grume',
 'guiro',
 'gummi',
 'gurge',
 'heigh',
 'heugh',
 'hewer',
 'hewgh',
 'hight',
 'hiree',
 'hirer',
 'hithe',
 'hiver',
 'homie',
 'homme',
 'hoove',
 'horme',
 'hough',
 'houri',
 'howre',
 'huger',
 'immew',
 'immit',
 'ither',
 'meith',
 'merer',
 'metho',
 'metre',
 'mhorr',
 'mieve',
 'mimeo',
 'mimer',
 'miter',
 'mitre',
 'mohur',
 'moire',
 'moove',
 'morro',
 'motet',
 'motte',
 'murre',
 'murri',
 'murti',
 'muter',
 'ogive',
 'oorie',
 'orgue',
 'ormer',
 'ortho',
 'ourie',
 'outre',
 'outro',
 'owrie',
 'reeve',
 'reggo',
 'regie',
 'regur',
 'rehem',
 'reive',
 'remet',
 'reorg',
 'rerig',
 'retem',
 'retie',
 'revet',
 'revie',
 'rewet',
 'rewth',
 'rheme',
 'rheum',


In [ ]:
from itertools import product, islice

def possible_flips():
    """Returns a generator of character 5-tuples where x represents grey,
    y yellow, and g green"""
    return product(*["xyg"]*5)

In [ ]:
def expectation(guess):
    return sum(
        probability(flip)
    )

('x', 'x', 'x', 'x', 'x')
('x', 'x', 'x', 'x', 'y')
('x', 'x', 'x', 'x', 'g')
('x', 'x', 'x', 'y', 'x')
('x', 'x', 'x', 'y', 'y')
('x', 'x', 'x', 'y', 'g')
('x', 'x', 'x', 'g', 'x')
('x', 'x', 'x', 'g', 'y')
('x', 'x', 'x', 'g', 'g')
('x', 'x', 'y', 'x', 'x')


In [121]:
from numpy import log2
from typing import Iterable

def match(regex) -> list[str]:
    """Finds all possible answers which match the given regex."""
    return re.findall(
        regex,
        answers,
        re.MULTILINE
    )


def match_regex(
        guess, flip, previous=None
    ) -> dict:
    """
    Takes a guess and flip and returns regex which can be used to match
    all compatable words.
    
    Also has the option to include a dictionary from a previous call to this
    function of the form

    {
        "pattern" : pattern - list[str]
        "yellows" : yellows - str
        "greys"   : greys   - str
    }.
    
    This will cause the function to return regex for matching the previous call
    as well.
    """

    if not previous:
        previous = {
            "pattern" : ["."]*5,
            "yellows" : "",
            "greys"   : ""  
        }

    pattern = previous["pattern"]
    yellows = previous["yellows"]
    greys = previous["greys"]

    for i, (char, f) in enumerate(zip(guess, flip)):
        if f == "x":
            greys += char

        elif f == "y":
            yellows += char
            if pattern[i] == ".":
                # add a negative match for `char`
                pattern[i] = f"[^{char}]"

            elif len(pattern[i]) > 1:
                # there's already a neg match, so add `char`
                pattern[i] = pattern[i][:-1] + char + "]"

        elif f == "g":
            pattern[i] = char

        # TODO - add support for letter repeats
        # .*y.*y works
    regex =  (
        "^"
        + "".join(f"(?=.*{y})" for y in yellows)  # lookahead: ensure yellows
        + (f"(?!.*[{greys}])" if greys else "")   # lookahead: ensure no greys
        + "".join(pattern)                        # match pattern
    )

    return {
        "regex" : regex,
        "pattern" : pattern,
        "greys": greys,
        "yellows": yellows
    }


def probability(flip, guess) -> float:
    """Calculates the probability of a flip given a guess."""
    # more principled approach would be a sum of probabilities
    return len(
            match(match_regex(guess, flip)["regex"])
        ) / len(answers)


def possible_flips():
    """Returns a generator of character 5-tuples where x represents grey,
    y yellow, and g green"""
    return product(*["xyg"]*5)


def flip_probabilites(guess) -> Iterable[tuple[tuple, float]]:
    """Calculates the probabilities of all possible flips given a guess.
    Returns an generator of (flip, probability) tuples."""
    return (
        (flip, probability(flip, guess))
        for flip in possible_flips()
    )


def expectation(guess) -> float:
    """Calculates the expectation (in bits of information) of a given guess."""
    return sum(
        - p * log2(p) if p != 0 else 0
        for _, p in flip_probabilites(guess)
    )

In [ ]:
test = lambda ys: f"(?={''.join(f'.*{y}' for y in ys)})"
test("abc")

'(?=.*a.*b.*c)'

In [117]:
guess1 = match_regex("crane", "xyxxy")
guess2 = match_regex("toils", "xxxxx", guess1)
guess3 = match_regex("brief", "xyxyy", guess2)
match(guess3["regex"])

['femur', 'ferry']

In [127]:
50 * 8000 / 1000

400.0

In [132]:
answer_list = answers.split()

In [138]:
r = re.compile(r"(?=c)[^c]r..e")
%timeit len(re.findall(r"(?=c)[^c]r..e", answers))

36.1 µs ± 802 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
33.2 µs ± 112 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [154]:
def get_pattern(answer, guess):

    pattern = ["x"]*5

    # mark greens
    for i in range(5):
        if answer[i] == guess[i]:
            pattern[i] = "g"
            answer = answer[:i] + "." + answer[i+1:]

    # mark yellows
    for i in range(5):
        if guess[i] in answer:
            pattern[i] = "y"
            answer = answer.replace(guess[i], ".", 1)

    return "".join(pattern)

In [156]:
from collections import Counter
from operator import itemgetter

answer_list = answers.split()
results = Counter(get_pattern(answer, "crane") for answer in answer_list)

sorted(results.items(), key=itemgetter(1), reverse=True)

[('xxxxx', 263),
 ('xxxxy', 153),
 ('xxyxx', 143),
 ('xyxxy', 143),
 ('xyxxx', 91),
 ('xyyxx', 76),
 ('xxxxg', 76),
 ('xxxyy', 66),
 ('yxxxx', 65),
 ('xxxyx', 59),
 ('xxyxg', 54),
 ('xxxgx', 54),
 ('xyyxy', 51),
 ('xxyyx', 50),
 ('xgxxx', 50),
 ('xxyxy', 48),
 ('xxgxx', 40),
 ('yxyxx', 32),
 ('xxgxg', 30),
 ('xyxyy', 29),
 ('xxgxy', 25),
 ('xgxxg', 25),
 ('xyxxg', 25),
 ('yxxxy', 24),
 ('yxxyx', 22),
 ('xxyyy', 21),
 ('xggxx', 21),
 ('xygxx', 20),
 ('xxygx', 18),
 ('xgxxy', 17),
 ('xgxgx', 17),
 ('xxggx', 16),
 ('xxxyg', 15),
 ('gxxxx', 15),
 ('yxgxx', 14),
 ('xyxyx', 13),
 ('xggxg', 12),
 ('yyxxx', 11),
 ('gxgxx', 11),
 ('ggxxy', 11),
 ('ggxxx', 11),
 ('yyxxy', 11),
 ('xxxgg', 11),
 ('xyyyx', 10),
 ('gxxxy', 10),
 ('gxxxg', 10),
 ('xygxy', 9),
 ('xxxgy', 9),
 ('xgyxy', 9),
 ('gxyxx', 9),
 ('gyxxx', 9),
 ('yyyxx', 8),
 ('xyyxg', 8),
 ('gyxxy', 8),
 ('xggyx', 7),
 ('ygxxx', 7),
 ('gxyxy', 7),
 ('gyyxx', 7),
 ('yxxxg', 7),
 ('yxxyg', 7),
 ('xxgyx', 7),
 ('xxyyg', 6),
 ('xgyxx', 6),
 ('yx

In [166]:
%timeit Counter(get_pattern(answer, "crane") for answer in answer_list)


3.74 ms ± 953 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [167]:
from bot import find_best_first_guess

In [169]:
%prun find_best_first_guess()

59.24805498123169 sec
      guess  bits info
8530  soare   5.816729
7642  roate   5.792096
7320  raile   5.747092
7868  salet   5.734364
6435  orate   5.731114
8403  slane   5.719685
7424  reast   5.713771
7852  saine   5.707397
7322  raine   5.705843
1439  carte   5.703036
8839  strae   5.690218
1424  caret   5.682820
1437  carse   5.675133
424   ariel   5.656729
9115  taser   5.654523
9060  taler   5.652818
1427  carle   5.648197
7845  saice   5.637505
9531  trine   5.629512
453   artel   5.629092
 

         97078760 function calls (97078567 primitive calls) in 59.301 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
 24670955   41.140    0.000   47.319    0.000 bot.py:22(get_pattern)
 24681612    6.372    0.000   53.691    0.000 bot.py:45(<genexpr>)
    10657    4.390    0.000   58.081    0.005 {built-in method _collections._count_elements}
 24671057    3.390    0.000    3.390    0.000 {method 'join' of 'str' objects}
 22048695    2.789    0.000    2.789    0.000 {method 'replace' of 'str' objects}
   876478    0.823    0.000    0.823    0.000 bot.py:47(<genexpr>)
    21315    0.157    0.000    0.980    0.000 {built-in method builtins.sum}
    10657    0.054    0.000   59.200    0.006 bot.py:40(expectation)
        1    0.038    0.038   59.238   59.238 bot.py:65(<listcomp>)
    10657    0.036    0.000   58.163    0.005 __init__.py:565(__init__)
    10657    0.017    0.000   58.126    0.005 __init__.py:640(update)
      

In [171]:
from random import choice

In [1]:
from bot import ALL_POSSIBLE_ANSWERS

In [3]:
"trove" in ALL_POSSIBLE_ANSWERS

True

In [1]:
%load_ext line_profiler

from bot import best_guess, expectation, ANSWERS
import numpy as np

def profile_expectation():
    for _ in range(10):
        mask = np.random.randint(0, 2, (len(ANSWERS))).astype(bool)
        best_guess(mask)

%lprun -f expectation profile_expectation()


/Users/ali/Documents/CS tomfoolery/wordle bot/bot.py:34: RuntimeWarning: divide by zero encountered in log2
  expected_info = np.sum(-np.log2(probabilities) * probabilities)
/Users/ali/Documents/CS tomfoolery/wordle bot/bot.py:34: RuntimeWarning: invalid value encountered in multiply
  expected_info = np.sum(-np.log2(probabilities) * probabilities)


Timer unit: 1e-06 s

Total time: 5.33153 s
File: /Users/ali/Documents/CS tomfoolery/wordle bot/bot.py
Function: expectation at line 25

Line #      Hits         Time  Per Hit   % Time  Line Contents
    25                                           def expectation(g, mask):
    26    129720     392532.0      3.0      7.4      n = np.count_nonzero(mask)
    27    129720    1493878.0     11.5     28.0      flips_from_guess = FLIPS[g][mask]
    28                                               # _, counts = np.unique(flips_from_guess, return_counts=True)
    29    129720     716093.0      5.5     13.4      counts = np.bincount(flips_from_guess)
    30    129720     405593.0      3.1      7.6      probabilities = counts /  n
    31                                               # TODO replace ^this  with a more flexible function
    32                                                   # which takes word frequency into account
    33                                           
    34    129720 

In [1]:
%load_ext line_profiler

from bot import best_guess, expectation, ANSWERS
import numpy as np

def profile_expectation():
    for _ in range(10):
        mask = np.random.randint(0, 2, (len(ANSWERS))).astype(bool)
        best_guess(mask)

%lprun -f expectation profile_expectation()


Timer unit: 1e-06 s

Total time: 5.53013 s
File: /Users/ali/Documents/CS tomfoolery/wordle bot/bot.py
Function: expectation at line 25

Line #      Hits         Time  Per Hit   % Time  Line Contents
    25                                           def expectation(g, mask):
    26    129720     432038.0      3.3      7.8      n = np.count_nonzero(mask)
    27    129720    1523426.0     11.7     27.5      possible_flips = FLIPS[g][mask]
    28    129720    1118665.0      8.6     20.2      probabilities = np.bincount(possible_flips) /  n
    29                                               # TODO replace ^this  with a more flexible function
    30                                                   # which takes word frequency into account
    31    129720     831233.0      6.4     15.0      infos = -np.log2(probabilities, where=(probabilities!=0))
    32                                           
    33    129720      67809.0      0.5      1.2      if g in ANSWER_SET:
    34               

In [13]:
arr = np.array([0, 1, 34])
arr = np.log2(arr, where=(arr!=0))
print(arr)
arr.sum()

[0.         0.         5.08746284]


5.087462841250339

In [8]:
set(open("words/answers.txt").readlines()) & set(open("words/guesses.txt").readlines())

set()

In [261]:
import pandas as pd

df = pd.read_csv("words/frequencies.csv", header=None)
df.columns = ["word", "frequency"]
df = df[df.word.str.len() == 5].reset_index(drop=True)
df.iloc[6000:6000+20]
# df

,word,frequency
6000,cleef,319350
6001,inane,319343
6002,ibsen,319327
6003,asccm,319248
6004,corky,318993
6005,cgcgg,318964
6006,yolks,318800
6007,dello,318654
6008,spook,318653
6009,amped,318536


In [ ]:
df.to_csv("words/5_f.csv")

In [13]:
import numpy as np
from bot import get_flip

In [15]:
get_flip("crane", "raked")

111

In [16]:
from bot import FLIPS

In [47]:
mask = np.random.randint(0, 2, size=FLIPS.shape[0]).astype(bool)
print(FLIPS[:,100])
print(mask)
print(FLIPS[:,100][mask])

[84 81 81 ... 81  0 81]
[False False False ...  True False  True]
[-94 -94 -66 ...  81  81  81]


In [61]:
from precomputing import GUESSES

In [101]:
WORD_LOOKUP = {w : i for i, w in enumerate(GUESSES)}

In [102]:
WORD_LOOKUP["crane"]

458

In [65]:
FLIPS = np.load("all_patterns.npy")

In [92]:
mask = ~(np.random.randint(0, 100, size=FLIPS.shape[1]).astype(bool))
print(mask)
expectation(458, mask)

[False False False ... False False False]


3.773557262275185

In [88]:
expectation(458, np.ones(FLIPS.shape[1], dtype=bool))


5.673254356256344

In [107]:
WORD_LOOKUP["brace"]

258

False

In [112]:
from itertools import product
PATTERN_LOOKUP = dict(enumerate("".join(pattern) for pattern in product(*["xyg"]*5)))

In [104]:
PATTERNS

{'xxxxx': 0,
 'xxxxy': 1,
 'xxxxg': 2,
 'xxxyx': 3,
 'xxxyy': 4,
 'xxxyg': 5,
 'xxxgx': 6,
 'xxxgy': 7,
 'xxxgg': 8,
 'xxyxx': 9,
 'xxyxy': 10,
 'xxyxg': 11,
 'xxyyx': 12,
 'xxyyy': 13,
 'xxyyg': 14,
 'xxygx': 15,
 'xxygy': 16,
 'xxygg': 17,
 'xxgxx': 18,
 'xxgxy': 19,
 'xxgxg': 20,
 'xxgyx': 21,
 'xxgyy': 22,
 'xxgyg': 23,
 'xxggx': 24,
 'xxggy': 25,
 'xxggg': 26,
 'xyxxx': 27,
 'xyxxy': 28,
 'xyxxg': 29,
 'xyxyx': 30,
 'xyxyy': 31,
 'xyxyg': 32,
 'xyxgx': 33,
 'xyxgy': 34,
 'xyxgg': 35,
 'xyyxx': 36,
 'xyyxy': 37,
 'xyyxg': 38,
 'xyyyx': 39,
 'xyyyy': 40,
 'xyyyg': 41,
 'xyygx': 42,
 'xyygy': 43,
 'xyygg': 44,
 'xygxx': 45,
 'xygxy': 46,
 'xygxg': 47,
 'xygyx': 48,
 'xygyy': 49,
 'xygyg': 50,
 'xyggx': 51,
 'xyggy': 52,
 'xyggg': 53,
 'xgxxx': 54,
 'xgxxy': 55,
 'xgxxg': 56,
 'xgxyx': 57,
 'xgxyy': 58,
 'xgxyg': 59,
 'xgxgx': 60,
 'xgxgy': 61,
 'xgxgg': 62,
 'xgyxx': 63,
 'xgyxy': 64,
 'xgyxg': 65,
 'xgyyx': 66,
 'xgyyy': 67,
 'xgyyg': 68,
 'xgygx': 69,
 'xgygy': 70,
 'xgygg': 71,
 '

In [120]:
%timeit expectation(258, mask)

28 µs ± 982 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [122]:
28 * 12000 / 1000000

0.336

In [133]:
import pandas as pd

In [136]:
def update_mask(g, flip, mask):
    return mask & (FLIPS[g] == flip)


In [140]:
from bisect import bisect_right

In [146]:
del GUESSES
from precomputing import GUESSES, ANSWERS
GUESSES = GUESSES + ANSWERS
i = bisect_right(answers, "crane")
GUESSES[i-2:i+2]

['crame', 'crams', 'crans', 'crape']

In [1]:
from bot import play_game

%prun play_game(answer="choke")

Guesses soare and narrowed the answer to one of 19 possibilities.
Guesses blink and narrowed the answer to one of 1 possibilities.
Guessed choke and won in 3 turns.
 

         1167632 function calls (1089800 primitive calls) in 1.177 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    25944    0.211    0.000    1.155    0.000 bot.py:25(expectation)
    25944    0.188    0.000    0.579    0.000 arraysetops.py:320(_unique1d)
155667/77835    0.142    0.000    0.879    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
    25944    0.126    0.000    0.144    0.000 function_base.py:1164(diff)
    25947    0.087    0.000    0.087    0.000 {method 'reduce' of 'numpy.ufunc' objects}
    25944    0.041    0.000    0.144    0.000 fromnumeric.py:69(_wrapreduction)
    25944    0.032    0.000    0.183    0.000 fromnumeric.py:2123(sum)
    25944    0.030    0.000    0.626    0.000 arraysetops.py:138(unique)
    25944    0.022    0.000    0.669    0.000 <__array_function__ internals>:2(unique)
        2    0.019    0.010    1.175    0.587 bot.py:75(<listcomp>)
    25944    0

In [8]:
import pandas as pd

df = pd.read_csv("words/unigram_freq.csv")
df = df[df.word.str.len() == 5]
df

,word,count
35,about,1226734006
45,other,978481319
56,which,810514085
57,their,782849411
62,there,701170205
...,...,...
333307,goovo,12711
333308,goova,12711
333320,goolh,12711
333328,gooek,12711


In [22]:
import numpy as np
df["prob"] = np.log(df["count"])
df[:4000]

,word,count,prob
35,about,1226734006,20.927621
45,other,978481319,20.701512
56,which,810514085,20.513179
57,their,782849411,20.478451
62,there,701170205,20.368261
...,...,...,...
33161,todas,672941,13.419413
33166,pinky,672775,13.419166
33169,mints,672742,13.419117
33178,webex,672246,13.418380


In [23]:
from precomputing import ANSWERS, GUESSES
from bisect import bisect_left

ANSWER_SET = set(
    bisect_left(GUESSES, answer) for
    answer in ANSWERS
    )



In [27]:
for g in GUESSES:
    try:
        j = ANSWERS.index("crane")
    except:
        continue
    
    if j not in ANSWER_SET:
        print(g, j)

In [28]:
'bling' in ANSWER_SET

False

In [29]:
len(ANSWERS)

2315

In [18]:
from bot import update_mask, best_guess, GUESSES, ANSWERS, PATTERN_LOOKUP
import numpy as np

mask = np.ones(FLIPS.shape[1]).astype(bool)

g = GUESSES.index("crane")
f = PATTERN_LOOKUP["gxygx"]
update_mask(g, f, mask)

# g = GUESSES.index("china")
# f = PATTERN_LOOKUP["gxxgy"]
# update_mask(g, f, mask)
best_guess(mask, print_best=10)
mask.any()

      index   word  expectation
0         0  aahed          0.0
8651   8651  pukey          0.0
8641   8641  puffy          0.0
8642   8642  puggy          0.0
8643   8643  pugil          0.0
8644   8644  puhas          0.0
8645   8645  pujah          0.0
8646   8646  pujas          0.0
8647   8647  pukas          0.0
8648   8648  puked          0.0


True

In [22]:
ANSWERS[376]

'china'